In [1]:
import os
import numpy as np
import threading as thr
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, MaxPool2D, Dense, Flatten, InputLayer, Activation, Dropout
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import SGD

In [2]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [3]:
mammo_o = []
mammo_f = []
label = []
project_folder = "../dataset/"
os.chdir(project_folder)
l = os.listdir()

In [4]:
def create_dataset(ls, o_img, f_img, lbl):
    for l in ls:
        if "_1_resized.pgm" in l:
            mo, mf = eng.dataset_filtered(eng.char(os.path.join(project_folder,l)), nargout = 2)
            o_img.append(mo)
            f_img.append(mf)
            lbl.append(1)
        elif "_2_resized.pgm" in l:
            mo, mf = eng.dataset_filtered(eng.char(os.path.join(project_folder,l)), nargout = 2)
            o_img.append(mo)
            f_img.append(mf)
            lbl.append(0)

In [5]:
os.chdir("./")
threads = []
chunk = 6
for i in range(49):
    t = thr.Thread(target = create_dataset, args = (l[i*chunk : (i+1)*chunk], mammo_o, mammo_f, label))
    threads.append(t)
    t.start()

for i in threads:
    i.join()

In [6]:
mammo_o = np.asarray(mammo_o, dtype = 'float32')/255.
mammo_f = np.asarray(mammo_f, dtype = 'float32')/255.
label = np.asarray(label)
mammo_o_4d = np.reshape(mammo_o, (147, 125, 125, 1))
print(mammo_o_4d.shape)
mammo_f_4d = np.reshape(mammo_f, (147, 64, 64, 1))
print(mammo_f_4d.shape)

(147, 125, 125, 1)
(147, 64, 64, 1)


In [7]:
def cnn_f(shape=(64, 64, 1)):
    model = Sequential([
        
        Conv2D(10, (3,3), padding = 'same', input_shape = shape),
        BatchNormalization(),
        Activation('relu'),
    
        MaxPool2D((4,4), strides = 2),
        #Dropout(0.1),
        
        
        Conv2D(16, (3,3), padding = 'same'),
        BatchNormalization(),
        Activation('relu'),
        
        MaxPool2D((4,4), strides = 2),
        #Dropout(0.1),
        
        
        Conv2D(29, (3,3), padding = 'same'),
        BatchNormalization(),
        Activation('relu'),
        
        MaxPool2D((4,4), strides = 2),
        #Dropout(0.1),
        
        Flatten(),
        
        Dense(10, activation = 'relu'),
        #Dropout(0.2),
        Dense(1, activation = 'sigmoid')        
        
    ])
    
    return model

In [8]:
model_f = cnn_f()
model_f.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 10)        100       
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 10)       40        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 64, 64, 10)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 10)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 31, 31, 16)        1456      
                                                                 
 batch_normalization_1 (Batc  (None, 31, 31, 16)       6

In [9]:
learning_rate = 0.001
model_f.compile(optimizer = SGD(learning_rate, momentum = 0.9), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [10]:
reduce_on_plateau = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0)

In [ ]:
X_train_f, X_val_f, Y_train_f, Y_val_f = train_test_split(mammo_f_4d, label, test_size = 0.2, random_state = 44)
batch_size = 21
train_f = model_f.fit(X_train_f, Y_train_f,
                            batch_size = batch_size,
                            epochs = 200,
                            verbose = 1,
                            validation_data = (X_val_f, Y_val_f),
                            callbacks = [reduce_on_plateau])

Epoch 1/200
6/6 [==============================] - 2s 137ms/step - loss: 0.8280 - accuracy: 0.6496 - val_loss: 0.6899 - val_accuracy: 0.5667 - lr: 0.0010
Epoch 2/200
6/6 [==============================] - 0s 70ms/step - loss: 0.6795 - accuracy: 0.6667 - val_loss: 0.6885 - val_accuracy: 0.5667 - lr: 0.0010
Epoch 3/200
6/6 [==============================] - 0s 42ms/step - loss: 0.6417 - accuracy: 0.6752 - val_loss: 0.6877 - val_accuracy: 0.5667 - lr: 0.0010
Epoch 4/200
6/6 [==============================] - 0s 49ms/step - loss: 0.6121 - accuracy: 0.6667 - val_loss: 0.6840 - val_accuracy: 0.5667 - lr: 0.0010
Epoch 5/200
6/6 [==============================] - 0s 45ms/step - loss: 0.5774 - accuracy: 0.6923 - val_loss: 0.6784 - val_accuracy: 0.5667 - lr: 0.0010
Epoch 6/200
6/6 [==============================] - 0s 46ms/step - loss: 0.5723 - accuracy: 0.6752 - val_loss: 0.6757 - val_accuracy: 0.5667 - lr: 0.0010
Epoch 7/200
6/6 [==============================] - 0s 42ms/step - loss: 0.5508 - 

6/6 [==============================] - 0s 42ms/step - loss: 0.4474 - accuracy: 0.7949 - val_loss: 0.7209 - val_accuracy: 0.6000 - lr: 1.0000e-07
Epoch 54/200
6/6 [==============================] - 0s 41ms/step - loss: 0.4372 - accuracy: 0.8120 - val_loss: 0.7209 - val_accuracy: 0.6000 - lr: 1.0000e-07
Epoch 55/200
6/6 [==============================] - 0s 42ms/step - loss: 0.4438 - accuracy: 0.8034 - val_loss: 0.7215 - val_accuracy: 0.6000 - lr: 1.0000e-07
Epoch 56/200
6/6 [==============================] - 0s 41ms/step - loss: 0.4468 - accuracy: 0.7949 - val_loss: 0.7220 - val_accuracy: 0.6000 - lr: 1.0000e-07
Epoch 57/200
6/6 [==============================] - 0s 47ms/step - loss: 0.4466 - accuracy: 0.7863 - val_loss: 0.7224 - val_accuracy: 0.6000 - lr: 1.0000e-07
Epoch 58/200
6/6 [==============================] - 0s 41ms/step - loss: 0.4588 - accuracy: 0.7863 - val_loss: 0.7225 - val_accuracy: 0.6000 - lr: 1.0000e-07
Epoch 59/200
6/6 [==============================] - 0s 41ms/step 

Epoch 105/200
6/6 [==============================] - 0s 50ms/step - loss: 0.4441 - accuracy: 0.8205 - val_loss: 0.6123 - val_accuracy: 0.6000 - lr: 1.0000e-10
Epoch 106/200
6/6 [==============================] - 0s 51ms/step - loss: 0.4456 - accuracy: 0.7949 - val_loss: 0.6111 - val_accuracy: 0.6000 - lr: 1.0000e-10
Epoch 107/200
6/6 [==============================] - 0s 51ms/step - loss: 0.4519 - accuracy: 0.8034 - val_loss: 0.6099 - val_accuracy: 0.6000 - lr: 1.0000e-10
Epoch 108/200
6/6 [==============================] - 0s 51ms/step - loss: 0.4468 - accuracy: 0.7863 - val_loss: 0.6091 - val_accuracy: 0.6000 - lr: 1.0000e-10
Epoch 109/200
6/6 [==============================] - 0s 51ms/step - loss: 0.4424 - accuracy: 0.8205 - val_loss: 0.6081 - val_accuracy: 0.6000 - lr: 1.0000e-10
Epoch 110/200
6/6 [==============================] - 0s 51ms/step - loss: 0.4400 - accuracy: 0.7863 - val_loss: 0.6070 - val_accuracy: 0.6333 - lr: 1.0000e-10
Epoch 111/200
6/6 [===========================

In [ ]:
acc = train_f.history['accuracy']
val_acc = train_f.history['val_accuracy']
loss = train_f.history['loss']
val_loss = train_f.history['val_loss']
    
epochs_range = range(1, len(acc)+1)
    #Train and validation accuracy 
plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
    #Train and validation loss 
plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
def cnn_f_aug(shape=(64, 64, 1)):
    model = Sequential([
        
        Conv2D(9, (3,3), padding = 'same', input_shape = shape),
        BatchNormalization(),
        Activation('relu'),
    
        MaxPool2D((4,4), strides = 2),
        #Dropout(0.1),
        
        
        Conv2D(22, (3,3), padding = 'same'),
        BatchNormalization(),
        Activation('relu'),
        
        MaxPool2D((4,4), strides = 2),
        #Dropout(0.1),
        
        
        Conv2D(33, (3,3), padding = 'same'),
        BatchNormalization(),
        Activation('relu'),
        
        MaxPool2D((4,4), strides = 2),
        #Dropout(0.1),
        
        Flatten(),
        
        Dense(10, activation = 'relu'),
        #Dropout(0.2),
        Dense(1, activation = 'sigmoid')        
        
    ])
    
    return model

In [ ]:
model_f_aug = cnn_f_aug()
model_f_aug.summary()

In [ ]:
learning_rate = 0.001
model_f_aug.compile(optimizer = SGD(learning_rate, momentum = 0.9), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
                rotation_range = 90,
                horizontal_flip = True,
                vertical_flip = True,
                validation_split = 0.20)

aug_train_f = aug.flow(mammo_f_4d, label, batch_size = 30, subset = 'training')
aug_val_f = aug.flow(mammo_f_4d, label, batch_size = 30, subset = 'validation')

In [ ]:
batch_size = 42
train_f_aug = model_f_aug.fit(aug_train_f,
                            batch_size = batch_size,
                            epochs = 200,
                            verbose = 1,
                            validation_data = aug_val_f,
                            callbacks = [reduce_on_plateau])

In [ ]:
acc = train_f_aug.history['accuracy']
val_acc = train_f_aug.history['val_accuracy']
loss = train_f_aug.history['loss']
val_loss = train_f_aug.history['val_loss']
    
epochs_range = range(1, len(acc)+1)
    #Train and validation accuracy 
plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
    #Train and validation loss 
plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
acc_f = []
acc_f_aug = []
for i in range(10):
    
    _, val_acc = model_f.evaluate(X_val_f, Y_val_f, verbose=0)
    print('Validation accuracy: %.3f' % (val_acc))
    acc_f.append(val_acc)

    _, val_acc = model_f_aug.evaluate(aug_val_f, verbose=0)
    print('Validation accuracy: %.3f' % (val_acc))
    acc_f_aug.append(val_acc)

In [ ]:
media_f = np.mean(acc_f)
std_f = np.std(acc_f)
print(media_f)
print(std_f)

In [ ]:
media_f_aug = np.mean(acc_f_aug)
std_f_aug = np.std(acc_f_aug)
print(media_f_aug)
print(std_f_aug)

In [ ]:
mammo_o_t, mammo_f_t, label_t = [], [], []
data_folder_t = "../test_dataset/"
os.chdir(data_folder_t)
l_t = os.listdir()

In [ ]:
def create_dataset_t(lista, o_img, f_img, labels):
    #Function calling the Matlab file in order to filter the images.
    for element in lista:
        if "_1_resized.pgm" in element:
            mo, mf = eng.dataset_filtered(eng.char(os.path.join(data_folder_t, element)), nargout = 2)
            o_img.append(mo)
            f_img.append(mf)
            labels.append(1)
        elif "_2_resized.pgm" in element:
            mo, mf = eng.dataset_filtered(eng.char(os.path.join(data_folder_t, element)), nargout = 2)
            o_img.append(mo)
            f_img.append(mf)
            labels.append(0)

In [ ]:
os.chdir("./")
threads = []
chunk = 6

for i in range(5):
    t = thr.Thread(target = create_dataset_t, args = (l_t[i*chunk : (i+1)*chunk], mammo_o_t, mammo_f_t, label_t))
    threads.append(t)
    t.start()
    
for j in threads:
    j.join()

In [ ]:
eng.quit()

In [ ]:
mammo_o_t = np.asarray(mammo_o_t, dtype = 'float32')/255.
mammo_f_t = np.asarray(mammo_f_t, dtype = 'float32')/255.
label_t = np.asarray(label_t)
#mammo_o_4d_t = np.reshape(mammo_o_t, (30, 125, 125, 1))
#print(mammo_o_4d_t.shape)
#mammo_f_4d_t = np.reshape(mammo_f_t, (30, 64, 64, 1))
#print(mammo_f_4d_t.shape)

In [ ]:
if media_f > media_f_aug:
    test_loss, test_acc = model_f.evaluate(mammo_f_t, label_t)
    preds_test = model_f.predict(mammo_f_t, verbose=1)
    fpr, tpr, _ = roc_curve(label_t, preds_test)
    roc_auc = auc(fpr, tpr)

    print('\n Test accuracy = %.3f'% (test_acc))
    print('\n AUC = %.3f'% (roc_auc))
else:
    test_loss, test_acc = model_f_aug.evaluate(mammo_f_t, label_t)
    preds_test = model_f_aug.predict(mammo_f_t, verbose=1)
    fpr, tpr, _ = roc_curve(label_t, preds_test)
    roc_auc = auc(fpr, tpr)
    
    print('\n Test accuracy = %.3f'% (test_acc))
    print('\n AUC = %.3f'% (roc_auc))

In [ ]:
test_loss, test_acc = model_test.evaluate(mammo_f_4d_t, label_t)
preds_test = model_test.predict(mammo_f_4d_t, verbose=1)
fpr, tpr, _ = roc_curve(label_t, preds_test)
roc_auc = auc(fpr, tpr)

print('\n Test accuracy = %.3f'% (test_acc))
print('\n AUC = %.3f'% (roc_auc))